In [12]:
import pandas as pd

# Load cleaned orders
orders = pd.read_csv("../../core/data/processed/orders_cleaned.csv")
orders["order_date"] = pd.to_datetime(orders["order_date"])

# Load customer CLV features
customer_clv = pd.read_csv("../../core/data/processed/customer_clv_features.csv")

orders.head(), customer_clv.head()


(   order_id  customer_id  product_id order_date  quantity  discount order_status
 0         1        15796        2346 2022-06-09         3      0.40    Delivered
 1         2          861         989 2022-05-15         4      0.42    Delivered
 2         3        38159         422 2023-10-26         3      0.02    Delivered
 3         4        44733        1945 2022-02-11         1      0.24    Delivered
 4         5        11285         878 2023-01-18         3      0.24    Delivered,
    customer_id  total_value  avg_monthly_value  value_std  active_months     trend  recency_months
 0            1         4.60           2.300000   1.951615              2 -2.760000               0
 1            2        13.86           1.980000   1.246916              7 -0.048571               0
 2            3        14.84           2.473333   1.005777              6  0.289143               0
 3            4         9.14           1.523333   1.184883              6  0.192000               1
 4     

In [13]:
customer_monthly = (
    orders
    .groupby([
        "customer_id",
        pd.Grouper(key="order_date", freq="M")
    ])
    .agg(
        monthly_revenue=("quantity", "sum"),
        avg_discount=("discount", "mean")
    )
    .reset_index()
)

customer_monthly.rename(columns={"order_date": "order_month"}, inplace=True)
customer_monthly.head()

C:\Users\yashm\AppData\Local\Temp\ipykernel_10656\1120461429.py:5: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  pd.Grouper(key="order_date", freq="M")


,customer_id,order_month,monthly_revenue,avg_discount
0,1,2022-02-28,4,0.08
1,1,2023-12-31,1,0.08
2,2,2022-02-28,4,0.12
3,2,2022-10-31,1,0.49
4,2,2022-12-31,1,0.39


In [14]:
customer_monthly = customer_monthly.merge(
    customer_clv,
    on="customer_id",
    how="left"
)

customer_monthly.head()


,customer_id,order_month,monthly_revenue,avg_discount,total_value,avg_monthly_value,value_std,active_months,trend,recency_months
0,1,2022-02-28,4,0.08,4.60,2.30,1.951615,2,-2.760000,0
1,1,2023-12-31,1,0.08,4.60,2.30,1.951615,2,-2.760000,0
2,2,2022-02-28,4,0.12,13.86,1.98,1.246916,7,-0.048571,0
3,2,2022-10-31,1,0.49,13.86,1.98,1.246916,7,-0.048571,0
4,2,2022-12-31,1,0.39,13.86,1.98,1.246916,7,-0.048571,0


In [15]:
customer_monthly["time_index"] = (
    customer_monthly
    .groupby("customer_id")["order_month"]
    .rank(method="dense")
    .astype(int)
)

customer_monthly.head()


,customer_id,order_month,monthly_revenue,avg_discount,total_value,avg_monthly_value,value_std,active_months,trend,recency_months,time_index
0,1,2022-02-28,4,0.08,4.60,2.30,1.951615,2,-2.760000,0,1
1,1,2023-12-31,1,0.08,4.60,2.30,1.951615,2,-2.760000,0,2
2,2,2022-02-28,4,0.12,13.86,1.98,1.246916,7,-0.048571,0,1
3,2,2022-10-31,1,0.49,13.86,1.98,1.246916,7,-0.048571,0,2
4,2,2022-12-31,1,0.39,13.86,1.98,1.246916,7,-0.048571,0,3


In [16]:
import os

output_path = "../../core/data/processed/customer_clv_forecast_ready.csv"
os.makedirs(os.path.dirname(output_path), exist_ok=True)

customer_monthly.to_csv(output_path, index=False)

print("✅ CLV forecasting prep file created")


✅ CLV forecasting prep file created
